In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from scipy.signal import lfilter
import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras import backend as K

from ss_perf_utils import *

np.random.seed(2)
torch.manual_seed(2)

ImportError: Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared 

In [2]:
def train_numpy(X,y,layer_dims,num_iters,lr,add_del=False):
    sigmoid = lambda z : 1./(1+np.exp(-z))
    
    din,dh,dout = tuple(layer_dims)
    m = X.shape[1]
    delta,prob,epsilon,max_hidden_size,tau = init_add_del()
    losses = []
    num_neurons = []
    
    W1 = np.random.randn(dh,din)
    b1 = np.random.randn(dh,1)
    W2 = np.random.randn(dout,dh)
    b2 = np.random.randn(dout,1)
    
    for t in range(num_iters):
        # Forwardprop
        Z1 = np.dot(W1,X)+b1
        A = Z1.clip(min=0) # relu
        Z2 = np.dot(W2,A)+b2
        yhat = sigmoid(Z2).clip(1e-6,1.-1e-6)
    
        loss = 1./m*(-np.dot(y,np.log(yhat).T)-np.dot(1-y,np.log(1-yhat).T))
        loss = loss.squeeze().item()
        losses.append(loss)
    
        # Backprop
        dyhat = -(np.divide(y,yhat) - np.divide(1-y, 1-yhat))
        dZ2 = dyhat*sigmoid(Z2)*(1-sigmoid(Z2))
        dW2 = 1./m*np.dot(dZ2,A.T)
        db2 = 1./m*np.sum(dZ2,1,keepdims=True)
        dA = np.dot(W2.T,dZ2)
        dZ1 = dA
        dZ1[Z1 < 0] = 0
        dW1 = 1./m*np.dot(dZ1,X.T)
        db1 = 1./m*np.sum(dZ1,1,keepdims=True)
    
        # gradient descent
        W1 -= lr*dW1
        b1 -= lr*db1
        W2 -= lr*dW2
        b2 -= lr*db2

        if add_del and t>tau:
            W1,b1,W2,b2 = delete_neurons_numpy(W1,b1,W2,b2,delta,prob)
            W1,b1,W2,b2 = add_neurons_numpy(W1,b1,W2,b2,losses,epsilon,delta,
                                              max_hidden_size,tau,prob)
        num_neurons.append(b1.shape[0])

        if 0==1:#t % max(1,num_iters // 20) == 0:
            print('loss after iteration %i: %f' % (t, losses[-1]))
            if add_del:
                print('# neurons after iteration %i: %d' % (t, num_neurons[-1]))
    
    return losses,num_neurons

In [3]:
def train_pytorch(X,y,layer_dims,num_iters,lr,device,dtype,add_del=False):
    sigmoid = lambda z : 1./(1+torch.exp(-z))
    
    din,dh,dout = tuple(layer_dims)
    m = X.shape[1]
    delta,prob,epsilon,max_hidden_size,tau = init_add_del()
    losses = []
    num_neurons = []
    
    W1 = torch.randn(dh, din, dtype=dtype, requires_grad=False, device=device)
    b1 = torch.randn(dh, 1, dtype=dtype, requires_grad=False, device=device)
    W2 = torch.randn(dout, dh, dtype=dtype, requires_grad=False, device=device)
    b2 = torch.randn(dout, 1, dtype=dtype, requires_grad=False, device=device)
    
    for t in range(num_iters):
        # Forwardprop
        Z1 = torch.mm(W1,X)+b1
        A = Z1.clamp(min=0) # relu
        Z2 = torch.mm(W2,A)+b2
        yhat = sigmoid(Z2).clamp(1e-6,1.-1e-6)
    
        criterion = nn.BCELoss()
        loss = criterion(yhat,y)
        loss = loss.squeeze_().item()
        losses.append(loss)
    
        # Backprop
        dyhat = -(torch.div(y,yhat) - torch.div(1-y, 1-yhat))
        dZ2 = dyhat*sigmoid(Z2)*(1-sigmoid(Z2))
        dW2 = 1./m*torch.mm(dZ2,A.t())
        db2 = 1./m*torch.sum(dZ2,1,keepdim=True)
        dA = torch.mm(W2.t(),dZ2)
        dZ1 = dA
        dZ1[Z1 < 0] = 0
        dW1 = 1./m*torch.mm(dZ1,X.t())
        db1 = 1./m*torch.sum(dZ1,1,keepdim=True)
    
        # gradient descent
        W1 -= lr*dW1
        b1 -= lr*db1
        W2 -= lr*dW2
        b2 -= lr*db2

        if add_del and t>tau:
            W1,b1,W2,b2 = delete_neurons_pytorch(W1,b1,W2,b2,delta,prob)
            W1,b1,W2,b2 = add_neurons_pytorch(W1,b1,W2,b2,losses,epsilon,delta,max_hidden_size,tau,prob,device)
        num_neurons.append(b1.shape[0])

        if 0==1:#t % max(1,num_iters // 20) == 0:
            print('loss after iteration %i: %f' % (t, losses[-1]))
            if add_del:
                print('# neurons after iteration %i: %d' % (t, num_neurons[-1]))
    
    return losses,num_neurons

In [16]:
def train_tensorflow(X,y,layer_dims,num_iters,lr):
    din,dh,dout = tuple(layer_dims)
    m = X.shape[0]
    delta,prob,epsilon,max_hidden_size,tau = init_add_del()
    losses = []
    num_neurons = []
    
    model = Sequential()
    model.add(Dense(dh, activation='relu', input_dim=din))
    model.add(Dense(dout, activation='sigmoid'))
    sgd = SGD(lr=lr, decay=1., momentum=0., nesterov=False)
    model.compile(loss='binary_crossentropy',optimizer=sgd)
    model.fit(X, y, epochs=num_iters, batch_size=y.shape[0], verbose=0)
    
    return losses,num_neurons

In [4]:
num_runs = 100
num_iters = 1000
num_samples = 10000
num_features = 2
num_hidden = 10
num_classes = 1
lr = 0.1
layer_dims = [num_features,num_hidden,num_classes]

X,y,x1,x2 = gen_data(samples=num_samples,var=0.01)

In [5]:
X_np = X.T
y_np = y.reshape(1,-1)

times_np = []
print('numpy starting')
for run in tqdm(range(num_runs)):
    tin = time.clock()
    losses,num_neurons = train_numpy(X_np,y_np,layer_dims,num_iters,lr,add_del=False)
    tout = time.clock()
    times_np.append(tout-tin)
print('numpy finished')

  0%|          | 0/100 [00:00<?, ?it/s]

numpy starting


100%|██████████| 100/100 [02:54<00:00,  1.74s/it]

numpy finished


In [7]:
device = torch.device('cpu')
dtype = torch.float

X_pt = torch.tensor(X,device=device,dtype=dtype).t()
y_pt = torch.tensor(y,device=device,dtype=dtype).reshape(1,-1)


times_pt_cpu = []
print('pytorch cpu starting')
for run in tqdm(range(num_runs)):
    tin = time.clock()
    losses,num_neurons = train_pytorch(X_pt,y_pt,layer_dims,num_iters,lr,device,dtype,add_del=False)
    tout = time.clock()
    times_pt_cpu.append(tout-tin)
print('pytorch cpu finished')

  0%|          | 0/100 [00:00<?, ?it/s]

pytorch cpu starting


100%|██████████| 100/100 [08:28<00:00,  5.08s/it]

pytorch cpu finished


In [9]:
device = torch.device('cuda:0')
dtype = torch.float

X_pt = torch.tensor(X,device=device,dtype=dtype).t()
y_pt = torch.tensor(y,device=device,dtype=dtype).reshape(1,-1)


times_pt_gpu = []
print('pytorch gpu starting')
for run in tqdm(range(num_runs)):
    tin = time.clock()
    losses,num_neurons = train_pytorch(X_pt,y_pt,layer_dims,num_iters,lr,device,dtype,add_del=False)
    tout = time.clock()
    times_pt_gpu.append(tout-tin)
print('pytorch gpu finished')

  0%|          | 0/100 [00:00<?, ?it/s]

pytorch gpu starting


100%|██████████| 100/100 [01:27<00:00,  1.14it/s]

pytorch gpu finished


In [17]:
X_tf = X
y_tf = y

times_tf = []
print('tensorflow starting')
for run in tqdm(range(num_runs)):
    tin = time.clock()
    losses,num_neurons = train_tensorflow(X_tf,y_tf,layer_dims,num_iters,lr)
    tout = time.clock()
    times_tf.append(tout-tin)
print('tensorflow finished')

  0%|          | 0/100 [00:00<?, ?it/s]

tensorflow starting


  9%|▉         | 9/100 [04:21<44:07, 29.09s/it]

KeyboardInterrupt: 

In [ ]:
#losses = np.array(losses)
#filt_neurons = lfilter([1.0/50]*50,1,num_neurons)
#filt_neurons[filt_neurons<1] = num_hidden

#plt.plot(losses,color='blue')
#plt.title('Loss')
#plt.show()

#plt.plot(filt_neurons,color='green')
#plt.title('# Neurons')
#plt.show()

UsageError: Line magic function `%conda` not found.


In [23]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9523508066745545685
]
